# Generate cell abundances and proportions for correlation matrices
Precursor code to correlation matrices in Figure 2/Extended Figure 2/Supplementary Figure 2

- Author: CW
- Last updated: 04/09/24

## 1. Load libraries

In [1]:
library(Seurat)

Attaching SeuratObject

‘SeuratObject’ was built with package ‘Matrix’ 1.6.4 but the current
version is 1.6.5; it is recomended that you reinstall ‘SeuratObject’ as
the ABI for ‘Matrix’ may have changed

Seurat v4 was just loaded with SeuratObject v5; disabling v5 assays and
validation routines, and ensuring assays work in strict v3/v4
compatibility mode



In [2]:
sessionInfo()

R version 4.2.3 (2023-03-15)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: CentOS Linux 7 (Core)

Matrix products: default
BLAS/LAPACK: /net/bmc-lab5/data/kellis/users/cbw3/conda/envs/r-kernel/lib/libopenblasp-r0.3.21.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] SeuratObject_5.0.1 Seurat_4.3.0.1    

loaded via a namespace (and not attached):
  [1] Rtsne_0.17             colorspace_2.1-0       deldir_2.0-4          
  [4] ggridges_0.5.6         IRdisplay_1.1          base64enc_0.1-3       
  [7] spatstat.data_3.0-4    leiden_0.4.3.1 

## 2. Load seurat object (rds file) containing scRNA seq of all 39 samples 

In [3]:
path_to_object = '/net/bmc-lab5/data/kellis/group/scCancer2023/objects_for_DC_manuscript/for_public/1_RNA_all.rds'
obj.RNA <- readRDS(path_to_object)
#remove tumor cells with no mp 

In [ ]:
obj.RNA <- subset(obj.RNA, subset = subtyped_cell_type %in% c('Tumor - NA', 'Tumor'), invert = TRUE)

In [ ]:
#also subset the patient 1122_1 out 
obj.RNA <- subset(obj.RNA, subset = patient_sample == '1122_1', invert = TRUE)

In [ ]:
obj <- obj.RNA

In [ ]:
unique(obj.RNA@meta.data[obj.RNA$reannotated_cell_type == 'CD4 T',]$subtyped_cell_type)

In [13]:
#saveRDS(obj.RNA, path_to_object)

'Plasma B''Macrophage M2''Endothelial''cDC2''Macrophage M1''Inflammatory fibroblast''DN NKT''Monocyte non-classical''pDC''other T''Monocyte classical''Macrophage tumor-associated''NK''c0(Texme)''c2(CD69+ Tex)''Myofibroblast''c4(HNRNPH1+ Tex) ''c8(HSPD1+ Tex)''c1(Tem)''c7(FTL+ Teff)''c5(GZMK+ Teff)''c3(Tn)''c9(GNLY+ Teff)''c6(NKT)''cDC1''Tumor''Cycling T''Keratinocyte''c4(Tex)''c0(Tn)''mDC''c1(Treg)''c3(Tm)''c2(CD69+ T)''Mast''c5(NKT)''Transitional B''Naive B''Memory B'

In [21]:
#CD8 subsets

oldCD8_names <- c('c0(Texme)','c1(Tem)', 'c2(CD69+ Tex)', 'c3(Tn)', 'c4(HNRNPH1+ Tex) ', 'c5(GZMK+ Teff)',
                  'c6(NKT)', 'c7(FTL+ Teff)','c8(HSPD1+ Tex)', 'c9(GNLY+ Teff)')
newCD8_names <- c('CD8 Texme', 'CD8 Tem', 'CD8 early activated Teff', 'CD8 Tn', 'CD8 HNRNPH1+ Teff', 'CD8 GZMK+ Teff', 'CD8 NKT', 
                  'CD8 FTL+ Tex', 'CD8 Tex/HS', 'CD8 GNLY+ Tex')

In [22]:
#change 
for (i in 1:length(oldCD8_names)) {
    obj@meta.data[obj@meta.data$subtyped_cell_type == oldCD8_names[i],]$subtyped_cell_type = newCD8_names[i]
}

ERROR: Error in `$<-.data.frame`(`*tmp*`, "subtyped_cell_type", value = "CD8 Texme"): replacement has 1 row, data has 0


In [12]:
oldCD4_names <- c('c0(Tn)', 'c1(Treg)', 'c2(CD69+ T)', 'c3(Tm)', 'c4(Tex)', 'c5(NKT)')
newCD4_names <- c('CD4 Tn', 'CD4 Treg', 'CD4 CD69+ T', 'CD4 Tm', 'CD4 Tex', 'CD4 NKT')

In [13]:
#change 
for (i in 1:length(oldCD4_names)) {
    obj@meta.data[obj@meta.data$subtyped_cell_type == oldCD4_names[i],]$subtyped_cell_type = newCD4_names[i]
}

In [20]:
saveRDS(obj, path_to_object)

In [5]:
dim(obj.RNA@meta.data[obj.RNA$reannotated_cell_type == 'Monocyte/Macrophage',])

dim(obj.RNA@meta.data[obj.RNA$subtyped_cell_type == 'Macrophage M2',])


[1] 13052    43

[1] 4506   43

In [21]:
#4506/13052

[1] 0.3452344

In [6]:
obj <- obj.RNA

### a. view object metadata

In [7]:
head(obj@meta.data)

,sample_ID,is_multiplet,cell_barcode,total_counts,percent_ribo,percent_heme,n_genes,compartment,general_cell_type,sample_ID_long,⋯,is_mDC_high,mDC_out_of_cDC,BRAF,NRAS,NF1,TERT,TP53.TP63,PTEN,MAP2K1.MAP3K2,response_short
,<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<int>,<fct>,<fct>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
11962_CELL_0,11962,no,CATCCGTCACCATTCC-1,56784,0.0560193,0,3384,Immune,B,D19-11962,⋯,1,0.208333333333333,Mut,0,0,0,0,0,0,R
11962_CELL_1,11962,no,CTGCTCACATGGTGGA-1,26671,0.1152188,0,4742,Immune,Myeloid,D19-11962,⋯,1,0.208333333333333,Mut,0,0,0,0,0,0,R
11962_CELL_2,11962,no,ATCACTTCAACAGCCC-1,21995,0.2357809,0,5097,Stromal,Endothelial,D19-11962,⋯,1,0.208333333333333,Mut,0,0,0,0,0,0,R
11962_CELL_3,11962,no,TCGCTCAAGGCACAAC-1,21838,0.2748878,0,4150,Immune,Myeloid,D19-11962,⋯,1,0.208333333333333,Mut,0,0,0,0,0,0,R
11962_CELL_4,11962,no,GTTACGATCTTCACAT-1,20354,0.2142576,0,5225,Stromal,Endothelial,D19-11962,⋯,1,0.208333333333333,Mut,0,0,0,0,0,0,R
11962_CELL_5,11962,no,GTTTGGACATCACGGC-1,19641,0.2227992,0,4925,Stromal,Endothelial,D19-11962,⋯,1,0.208333333333333,Mut,0,0,0,0,0,0,R


In [8]:
#view column names of the object. note that "compartment", "general_cell_type",
#"reannotated_cell_type", 'subtyped_cell_type' are all columns
colnames(obj@meta.data)

[1] "sample_ID"             "is_multiplet"          "cell_barcode"         
 [4] "total_counts"          "percent_ribo"          "percent_heme"         
 [7] "n_genes"               "compartment"           "general_cell_type"    
[10] "sample_ID_long"        "patient"               "uid"                  
[13] "age"                   "sex"                   "subtype"              
[16] "treatment_group"       "prior_therapy"         "state"                
[19] "response"              "tissue"                "day"                  
[22] "pfs"                   "nCount_RNA"            "nFeature_RNA"         
[25] "percent_mt"            "nCount_SCT"            "nFeature_SCT"         
[28] "S.Score"               "G2M.Score"             "Phase"                
[31] "reannotated_cell_type" "subtyped_cell_type"    "patient_sample"       
[34] "is_mDC_high"           "mDC_out_of_cDC"        "BRAF"                 
[37] "NRAS"                  "NF1"                   "TERT"                 
[40] "TP53.TP63"             "PTEN"                  "MAP2K1.MAP3K2"        
[43] "response_short"

### b. plot object by different cell type categories

In [ ]:
#look at the cell types of the object 
options(repr.plot.height = 10, repr.plot.width = 17)

p1 <- DimPlot(obj, reduction = "umap",group.by = 'subtyped_cell_type', pt.size = 1, label = FALSE, ncol = 2) 

p2 <- DimPlot(obj, reduction = "umap",group.by = 'reannotated_cell_type', pt.size = 1, label = FALSE) 

p3 <- DimPlot(obj, reduction = "umap", group.by = 'general_cell_type',pt.size = 1, label = FALSE)

p4 <- DimPlot(obj, reduction = "umap", group.by = 'compartment', pt.size = 1,label = FALSE)

p1+ p2 + p3 + p4

## 3. Retrieve sample-level counts by compartment, general cell type, reannotated cell type, and subtyped cell type

### a. run get_counts for each category of cell type 

In [24]:
colnames(obj.RNA@meta.data)

[1] "sample_ID"             "is_multiplet"          "cell_barcode"         
 [4] "total_counts"          "percent_ribo"          "percent_heme"         
 [7] "n_genes"               "compartment"           "general_cell_type"    
[10] "sample_ID_long"        "patient"               "uid"                  
[13] "age"                   "sex"                   "subtype"              
[16] "treatment_group"       "prior_therapy"         "state"                
[19] "response"              "tissue"                "day"                  
[22] "pfs"                   "nCount_RNA"            "nFeature_RNA"         
[25] "percent_mt"            "nCount_SCT"            "nFeature_SCT"         
[28] "S.Score"               "G2M.Score"             "Phase"                
[31] "reannotated_cell_type" "subtyped_cell_type"    "patient_sample"       
[34] "is_mDC_high"           "mDC_out_of_cDC"

In [13]:
get_counts <- function(obj, category, path_to_save) {
    samples <- unique(obj@meta.data$patient_sample)
    
    if(category == 'compartment') {
        cell_types <- as.character(unique(obj@meta.data$compartment))
    }
    if(category == 'general_cell_type') {
        cell_types <- as.character(unique(obj@meta.data$general_cell_type))
    }
    if(category == 'reannotated_cell_type') {
        cell_types <- as.character(unique(obj@meta.data$reannotated_cell_type))
    }
    if(category == 'subtyped_cell_type') {
        cell_types <- as.character(unique(obj@meta.data$subtyped_cell_type))
    }
    
    #save each cell type specific object as loom

    #dataframe to store absolute cell counts for every sample
    all_cell_abundance <- data.frame(matrix(NA,    # Create empty data frame
                              nrow = length(cell_types) + 1,
                              ncol = length(samples)))
    rownames(all_cell_abundance) = append(cell_types, 'Total')
    colnames(all_cell_abundance) = samples

    for(sample in samples) {    

        #total number of cells in sample
        total_cells_in_sample = dim(obj@meta.data[obj@meta.data$patient_sample == sample,])[1]

        ##get number of cells present for each subtyped cell type 
        cell_abundance = c()
        for (ct in cell_types) {
            #get the number of cells present 
            celltype_abundance_in_sample = dim(obj@meta.data[obj@meta.data[category] == ct & obj@meta.data$patient_sample == sample,])[1]
            cell_abundance = append(cell_abundance, celltype_abundance_in_sample)
        #add cell_abundance as column to dataframe

        #add total abundance 
            }
        cell_abundance = append(cell_abundance, dim(obj@meta.data[obj@meta.data$patient_sample == sample,])[1])
        all_cell_abundance[sample] = cell_abundance
        }
    #save all_cell_abundance
    write.csv(all_cell_abundance, paste0(path_to_save, category, '_abundance_all_samples_2.csv'))

    return(all_cell_abundance)

    
}

In [9]:
#####TODO#### 

#replace path with directory of where you will save your files 
path_to_save = '/home/cbw3/data/figure_generation/Figure3/abundance_and_proportions/'

In [15]:
#run to generate the csv files of the counts 
get_counts(obj, 'compartment', path_to_save)
get_counts(obj, 'general_cell_type', path_to_save)
reannotated_counts <- get_counts(obj, 'reannotated_cell_type', path_to_save)
subtyped_counts <- get_counts(obj, 'subtyped_cell_type', path_to_save)

,1098_5,1098_4,1098_3,1063_1,946_1,926_1,406_3,978_1,98_1,964_1,⋯,776_1,856_1,808_1,1098_1,1224_1,1224_2,406_2,1098_2,452_1,620_2
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
Immune,2502,958,1463,303,221,4505,5531,229,801,562,⋯,4433,1470,464,1295,448,2544,1094,463,2907,5687
Stromal,292,437,439,2,135,237,115,100,367,148,⋯,85,405,172,274,426,3174,191,31,36,45
Tumor,0,367,238,1529,197,118,290,3493,1727,706,⋯,0,1241,1435,1327,0,0,210,909,5891,304
Total,2794,1762,2140,1834,553,4860,5936,3822,2895,1416,⋯,4518,3116,2071,2896,874,5718,1495,1403,8834,6036


,1098_5,1098_4,1098_3,1063_1,946_1,926_1,406_3,978_1,98_1,964_1,⋯,776_1,856_1,808_1,1098_1,1224_1,1224_2,406_2,1098_2,452_1,620_2
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
B,3,2,4,20,1,1167,1196,27,4,4,⋯,315,42,13,0,0,20,10,27,6,26
Myeloid,236,234,348,162,166,344,487,94,330,194,⋯,156,350,48,353,45,395,1058,334,1448,499
Endothelial,239,34,281,1,63,160,59,23,155,113,⋯,77,343,97,150,35,230,17,10,2,7
Fibroblastic,53,391,142,0,70,77,56,77,136,26,⋯,7,62,75,124,167,1256,174,21,31,38
NK/T (CD8+),2263,529,746,96,33,2801,3810,26,237,357,⋯,1408,800,322,524,201,1099,12,31,1317,4739
Tumor,0,367,238,1529,197,118,290,3493,1727,706,⋯,0,1241,1435,1327,0,0,210,909,5891,304
Keratinocyte,0,12,16,1,2,0,0,0,76,9,⋯,1,0,0,0,224,1688,0,0,3,0
NK/T (CD4+),0,185,356,25,16,193,37,80,222,7,⋯,2554,277,80,409,200,918,13,69,130,423
Mast,0,8,9,0,5,0,1,2,8,0,⋯,0,1,1,9,2,112,1,2,6,0


In [41]:
sum(t(subtyped_counts)[,'Macrophage M2'])

[1] 4506

In [39]:
sum(t(reannotated_counts)[,'Monocyte/Macrophage'])

[1] 13052

check that you see four csv files in your path with the names category + '_abundance_all_samples.csv'

## b. create a dataframe that maps every category of cell type to each other

In [10]:
subtyped_cell_types = unique(obj.RNA@meta.data$subtyped_cell_type)

In [11]:
cell_type_mapping <- data.frame(matrix(nrow = length(subtyped_cell_types), ncol = 4, ))
colnames(cell_type_mapping) <- c("subtype", "reannotated", "general", 'compartment')

ind = 1

for (ct in subtyped_cell_types){
    
    reannotated_cell_type = as.character(obj@meta.data[obj@meta.data$subtyped_cell_type == ct,]$reannotated_cell_type[1])

    general_cell_type = as.character(obj@meta.data[obj@meta.data$subtyped_cell_type == ct,]$general_cell_type[1])
    
    compartment = as.character(obj@meta.data[obj@meta.data$subtyped_cell_type == ct,]$compartment[1])

    mapping <- c(ct, reannotated_cell_type, general_cell_type, compartment)
    
    #add mapping to cell_type_mapping
    
    cell_type_mapping[ind,] = mapping
    
    ind <- ind + 1
    }

In [12]:
cell_type_mapping

subtype,reannotated,general,compartment
<chr>,<chr>,<chr>,<chr>
Plasma B,B,B,Immune
Macrophage M2,Monocyte/Macrophage,Myeloid,Immune
Endothelial,Endothelial,Endothelial,Stromal
cDC1,cDC,Myeloid,Immune
Macrophage M1,Monocyte/Macrophage,Myeloid,Immune
Inflammatory fibroblast,Fibroblast,Fibroblastic,Stromal
cDC2,cDC,Myeloid,Immune
DN NKT,Other T,NK/T (CD8+),Immune
Monocyte non-classical,Monocyte/Macrophage,Myeloid,Immune


In [40]:
#### TODO #### 
#view the dataframe. check that all mappings are unique (can search for function that may check this)
write.csv(cell_type_mapping, '/home/cbw3/data/figure_generation/Figure3/celltype_mapping.csv')

# 4. retrieve proportions in relation to different compartments 

### a. load previously generated csvs of the counts for each category 

In [23]:
path_to_save

[1] "/home/cbw3/data/figure_generation/Figure3/abundance_and_proportions/"

In [13]:
subtype_counts <- read.csv(paste0(path_to_save, 'subtyped_cell_type_abundance_all_samples_2.csv'), row.names = 1,header = TRUE, 
                              check.names = FALSE)

reannotated_counts <- read.csv(paste0(path_to_save, 'reannotated_cell_type_abundance_all_samples_2.csv'), row.names = 1, header = TRUE, 
                          check.names = FALSE)

general_counts <- read.csv(paste0(path_to_save, 'general_cell_type_abundance_all_samples_2.csv'), row.names = 1, header = TRUE, 
                          check.names = FALSE)

compartment_counts <- read.csv(paste0(path_to_save, 'compartment_abundance_all_samples_2.csv'), row.names = 1,header = TRUE, 
                              check.names = FALSE)

### b. get compartment/total proportions and save

In [14]:
#Initialize a dataframe to store the proportions 
proportion_df <- data.frame(matrix(ncol = length(unique(obj@meta.data$uid)), nrow = 0))
colnames(proportion_df) <- unique(obj@meta.data$patient_sample)

for(ct in rownames(compartment_counts)) {
        
    cell_types_for_sample <- cell_type_mapping[cell_type_mapping$compartment == ct,]
    
    #divide compartment row by the total 
    compartment_row = compartment_counts[rownames(compartment_counts) == ct,]
    all_row = compartment_counts[rownames(compartment_counts) == 'Total',]
    
    #
    compartment_all <- compartment_row/all_row
    rownames(compartment_all) = paste0(ct, '/Total')
    
    #append to dataframe 
    proportion_df = rbind(proportion_df,compartment_all)
    
}

#save the proportions
write.csv(proportion_df, paste0(path_to_save, 'compartment_proportions_all_samples_2.csv'))

### c. get general cell type/compartment and general cell type/total proportions

In [15]:
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.3     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [16]:
#Initialize a dataframe to store the proportions 
proportion_df <- data.frame(matrix(ncol = length(unique(obj@meta.data$uid)), nrow = 0))
colnames(proportion_df) <- unique(obj@meta.data$patient_sample)

for(ct in unique(cell_type_mapping$general)) {
        
    cell_types_for_sample <- cell_type_mapping[cell_type_mapping$general == ct,]
    mapping <- cell_type_mapping[cell_type_mapping$general == ct,][,c('general', 'compartment')]
    mapping <- distinct(mapping)

    #divide rows by each other 
    general_row = general_counts[rownames(general_counts) == mapping$general,]
    compartment_row = compartment_counts[rownames(compartment_counts) == mapping$compartment,]
    all_row = compartment_counts[rownames(compartment_counts) == 'Total',]
    
    #
    general_compart <- general_row/compartment_row
    rownames(general_compart) = paste0(ct, '/', mapping$compartment)
    
    general_all <- general_row/all_row
    rownames(general_all) = paste0(ct, '/Total')
    
    
    #append to dataframe 
    proportion_df = rbind(proportion_df,general_compart)
    proportion_df = rbind(proportion_df,general_all)
    
}

#proportion_df
#save the proportions
write.csv(proportion_df, paste0(path_to_save, 'general_cell_type_proportions_all_samples_2.csv'))

### d. get reannotated/general, reannotated/compartment, and reannotated/total proportions

In [46]:
cell_type_mapping[cell_type_mapping$reannotated == ct,]


,subtype,reannotated,general,compartment
,<chr>,<chr>,<chr>,<chr>
40,Mast,Mast,Mast,Immune


In [17]:
#Initialize a dataframe to store the proportions 
proportion_df <- data.frame(matrix(ncol = length(unique(obj@meta.data$uid)), nrow = 0))
colnames(proportion_df) <- unique(obj@meta.data$patient_sample)

#get reannotated proportions
for(ct in unique(obj@meta.data$reannotated_cell_type)) {
        
    cell_types_for_sample <- cell_type_mapping[cell_type_mapping$reannotated == ct,]
    mapping <- cell_type_mapping[cell_type_mapping$reannotated == ct,][,c('reannotated', 'general', 'compartment')]
    mapping <- distinct(mapping)
    
    #divide rows by each other 
    reannotated_row = reannotated_counts[rownames(reannotated_counts) == mapping$reannotated,]
    general_row = general_counts[rownames(general_counts) == mapping$general,]
    compartment_row = compartment_counts[rownames(compartment_counts) == mapping$compartment,]
    
    all_row = compartment_counts[rownames(compartment_counts) == 'Total',]
    
    #
    reannot_general <- reannotated_row/general_row
    rownames(reannot_general) = paste0(ct, '/', mapping$general)
    
    if(mapping$general != mapping$compartment) {
        reannot_compart <- reannotated_row/compartment_row
        rownames(reannot_compart) = paste0(ct, '/', mapping$compartment)
    }

    reannot_all <- reannotated_row/all_row
    rownames(reannot_all) = paste0(ct, '/Total')
    
    
    #add to proportion 
    proportion_df = rbind(proportion_df,reannot_general)
    proportion_df = rbind(proportion_df,reannot_compart)
    proportion_df = rbind(proportion_df,reannot_all)
    
}

#proportion_df
#save the proportions
write.csv(proportion_df, paste0(path_to_save, 'reannotated_cell_type_proportions_all_samples_2.csv'))

### e. get subtyped/reannotated, subtyped/general, subtpyed/compartment, and subtyped/total proportions

In [18]:
#Initialize a dataframe to store the proportions 
proportion_df <- data.frame(matrix(ncol = length(unique(obj@meta.data$uid)), nrow = 0))
colnames(proportion_df) <- unique(obj@meta.data$patient_sample)

for(ct in unique(obj.RNA$subtyped_cell_type)) {
    
    cell_types_for_sample <- cell_type_mapping[cell_type_mapping$subtype == ct,]
    mapping <- cell_type_mapping[cell_type_mapping$subtype == ct,]
    
    #divide rows by each other 
    subtype_row = subtype_counts[rownames(subtype_counts) == ct,]
    reannotated_row = reannotated_counts[rownames(reannotated_counts) == mapping$reannotated,]
    general_row = general_counts[rownames(general_counts) == mapping$general,]
    compartment_row = compartment_counts[rownames(compartment_counts) == mapping$compartment,]
    all_row = compartment_counts[rownames(compartment_counts) == 'Total',]
    
    #
    subtype_reannot <- subtype_row/reannotated_row
    subtype_general <- subtype_row/general_row
    subtype_compart <- subtype_row/compartment_row
    subtype_all <- subtype_row/all_row
    
    # put all rows together 
    subtype_reannot <- subtype_row/reannotated_row
    rownames(subtype_reannot) = paste0(ct, '/', mapping$reannotated)
    
    proportion_df = rbind(proportion_df,subtype_reannot)

    
    
    if(mapping$general != mapping$reannotated) {
        subtype_general <- subtype_row/general_row
        rownames(subtype_general) = paste0(ct, '/', mapping$general)
        proportion_df = rbind(proportion_df,subtype_general)

    }
    
    if(mapping$compartment != mapping$general){

    subtype_compartment <- subtype_row/compartment_row
    rownames(subtype_compartment) = paste0(ct, '/', mapping$compartment)
    #add to proportion df 
    proportion_df = rbind(proportion_df,subtype_compartment)
    }
    
    subtype_all <- subtype_row/all_row
    rownames(subtype_all) = paste0(ct, '/Total')
    proportion_df = rbind(proportion_df,subtype_all)

}

#proportion_df
#save the proportions
write.csv(proportion_df, paste0(path_to_save, 'subtyped_cell_type_proportions_all_samples_2.csv'))

check that you have 4 files with proportions. you should have generated 8 total csvs from this notebook. Check that the files look okay (no missing values, proportions are all < 1). The subtyped cell proportions file should have the most number of rows, followed by the reannotated, general, and compartment proportions. 

you can run simple tests to check the values of dataframes (will also be in the next notebook). 

Once you have checked the files, you have finished this part!